# 1. PRODUCT WEB CRAWLING
1.1. Brand \
1.2. Main image of product \
1.3. Detailed information 

## 1.1. Brand

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import json
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re
import time
import datetime

In [3]:
# Set the path to the ChromeDriver executable
path  = r"C:\Users\NT550\Downloads\chromedriver_win32 (1)\chromedriver.exe"
s = Service(path)
driver = webdriver.Chrome(service=s)
driver.get('https://www.musinsa.com/brands')
time.sleep(2)
brand = []
for page in range(3, 13):
    time.sleep(1)
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/section/div[3]/div/div[2]/div/div/a[{0}]'.format(page)).send_keys(Keys.ENTER)
    for i in range(1, 101):
        name = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/section/div[2]/div/ul/li[{0}]/dl/dt/a'.format(i)).text
        name_eng = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/section/div[2]/div/ul/li[{0}]/dl/dd/a'.format(i)).text.split(sep = "(")[0].rstrip()
        brand.extend([name, name_eng])

In [6]:
print(len(brand))
total_brand = []
total_brand.extend(brand)
print(total_brand)

2000

## 1.2. Main image of product

In [ ]:
# function to download image
def download_image(url, save_path):
    response = requests.get(url)
    response.raise_for_status()
    with open(save_path, 'wb') as file:
        file.write(response.content)
file_path = r"C:\Users\z1one\OneDrive\바탕 화면\Portfolio_desktop\3_Utilization-of-Image-Reviews-from-Online-Shopping-Mall\DATA\ProductNum.xlsx"
category_list = pd.read_excel('ProductNum.xlsx',index_col=False)

# get main image of goods
def get_main_img(goods_id):
    # Count the number of photos
    k = 0
    # Get the product URL
    url = f"https://www.musinsa.com/app/goods/{goods_id}"
    # Use BeautifulSoup
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:
        # Count the number of main photos
        count = len(soup.find_all("ul", {"class": "product_thumb"})[0].find_all("img"))

        # Create a product folder
        os.mkdir(str(goods_id))

        # Get images
        for i in range(count):
            # Name for each photo (ProductID_PhotoCount)
            temp = str(goods_id) + "_" + str(k)

            # Get the thumbnail image URL
            url1 = soup.find_all("ul", {"class": "product_thumb"})[0].find_all("img")[i].get("src")

            # Replace thumbnail image _60 with _500 using regular expression
            url2 = re.sub(r'(?<=\d{14}_)\d+(?=\.jpg)', '500', url1)
            file_url = "https:" + url2

            # Photo save path
            save_path = f"{goods_id}/{temp}(jpg).jpg"

            # Photo download function
            download_image(file_url, save_path)

            # Count
            k += 1
            success_list = [goods_id, 1]
    except:
        fail_list = [goods_id, 0]
        return fail_list
    return success_list


In [ ]:
s = Service(path)

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode (without a visible browser window)

# Initialize the Chrome driver
driver = webdriver.Chrome(service=s)

# List of goods IDs (example)
goods_ids = [1848166, 3105468, 3324814, 3134702]

# List to store completed IDs
done_list = []

# Run the get_main_img function for each goods_id
for goods_id in goods_ids:
    done_id = get_main_img(goods_id)  # Assuming you have defined and implemented the get_main_img function
    done_list.append(done_id)

# Convert the results to a DataFrame
id_list = pd.DataFrame(done_list, columns=["goods_id", "result"])

# Set the path for saving the DataFrame to a CSV file
id_list.to_csv('img_list.csv')

# Read a CSV file named "review/review_4000.csv" into a DataFrame
review = pd.read_csv("review/review_4000.csv", index_col=False)

## 1.3. Detailed information

In [ ]:
def item_info(id):
    url = "https://www.musinsa.com/app/goods/"+ str(id) + "/0"

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36", "Accept-Language": "ko-kr,ko"}
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    
    item_div = soup.find_all('div', class_= "right_contents section_product_summary")
    
    # product_info
		
    for con in item_div:
        if len(con.find('a', href = "https://www.musinsa.com/categories/item/001")) !=0:
            cat1 = con.find('a', href = "https://www.musinsa.com/categories/item/001").text

            item['cat1'] = cat1
        if len(con.find('a', href = "https://www.musinsa.com/categories/item/001002")) !=0:
            cat2 = con.find('a', href = "https://www.musinsa.com/categories/item/001002").text.split('/')[1]
            item['cat2'] = cat2
        print(cat1)
        print(cat2)
        
    # product_titles
        if len(con.find('span', class_="product_title")) != 0:
            product_title = con.find('span', class_="product_title").text.strip()
            item['product_title'] = product_title
        print(product_title)
    
    # brands, item num, season, gender
        if len(con.find("div", "explan_product product_info_section").find('a'))!=0:
            # info = con.find("div", "explan_product product_info_section").text.strip('').split('\n')
            # info =list(filter(None, info))
            brand = con.find("div", "explan_product product_info_section").find('a').text
            item['brand'] = brand
        if len(con.find("div", "explan_product product_info_section").find('p', class_= "product_article_contents").find('strong'))!=0:
            item_num = con.find("div", "explan_product product_info_section").find('p', class_= "product_article_contents").find('strong').text.split('/')[1].strip(' ')
            item['item_num'] = item_num
        print(brand)
        print(item_num)
        
    # season, gender

        if len(con.find("div", "explan_product product_info_section").find_all('p', class_= "product_article_contents")[1].find('strong'))!=0:
            season = con.find("div", "explan_product product_info_section").find_all('p', class_= "product_article_contents")[1].find('strong').text.split()
            season = ' '.join(season)
            item['season'] = season
        if len(con.find("div", "explan_product product_info_section").find_all('p', class_= "product_article_contents")[1].find('span', class_= "txt_gender"))!=0:
            gender = con.find("div", "explan_product product_info_section").find_all('p', class_= "product_article_contents")[1].find('span', class_= "txt_gender").text.strip()
            item['gender'] = gender
        print(season)
        print(gender)
        
    # likes, rates, tags
        if len(con.find("div", "explan_product product_info_section").find('span', class_="prd-score__rating")) != 0:
            rate= con.find("div", "explan_product product_info_section").find('span', class_="prd-score__rating").text

            item['rate'] = con.find("div", "explan_product product_info_section").find('span', class_="prd-score__rating").text
        if len(con.find("div", "explan_product product_info_section").find('li', class_= "product_section_like").find('span', class_="prd_like_cnt")) != 0:
            likes = con.find("div", "explan_product product_info_section").find('li', class_= "product_section_like").find('span', class_="prd_like_cnt").text
            item['likes'] = likes
        if len(con.find("div", "explan_product product_info_section").find('li', class_='article-tag-list list').find('p', class_= "product_article_contents")) != 0:
            tags = con.find("div", "explan_product product_info_section").find('li', class_='article-tag-list list').find('p', class_= "product_article_contents").text.split('\n')
            tags = ''.join(tags)
            item['tags'] = tags
        print(rate)
        print(likes)
        print(tags)
        
    # color
        # colors = []
        # cols = con.find('ul', class_='prd-group').find_all('p', class_='n-tooltip tooltip-musinsa tool-bottom')
		# 		if len(col) !=0:        
		# 			for col in cols:
        #    c = col.text.strip(' \n')
        #    colors.append(c)

        # print(colors)